In [14]:
# Import dependencies
import pandas as pd
import math

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read data
plays = pd.read_csv('https://storage.googleapis.com/big-data-bowl/plays.csv')
players = pd.read_csv('https://storage.googleapis.com/big-data-bowl/players.csv')

week1 = pd.read_csv('https://storage.googleapis.com/big-data-bowl/week1.csv')
# week2 = pd.read_csv('https://storage.googleapis.com/big-data-bowl/week2.csv')
# week3 = pd.read_csv('https://storage.googleapis.com/big-data-bowl/week3.csv')
# week4 = pd.read_csv('https://storage.googleapis.com/big-data-bowl/week4.csv')
# week5 = pd.read_csv('https://storage.googleapis.com/big-data-bowl/week5.csv')
# week6 = pd.read_csv('https://storage.googleapis.com/big-data-bowl/week6.csv')
# week7 = pd.read_csv('https://storage.googleapis.com/big-data-bowl/week7.csv')
# week8 = pd.read_csv('https://storage.googleapis.com/big-data-bowl/week8.csv')

tracking = week1
#.append([
#    week2,
#    week3,
#    week4,
#    week5,
#    week6,
#    week7,
#    week8,
#])

In [3]:
# Find tracking possession
player_tracking = tracking[tracking['team'] != 'football']
player_tracking = player_tracking.merge(players[['nflId', 'officialPosition']])
player_tracking = player_tracking.merge(plays[['gameId', 'playId', 'possessionTeam']])
player_tracking['possession'] = player_tracking['team'] == player_tracking['possessionTeam']
player_tracking

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,officialPosition,possessionTeam,possession
0,2021090900,97,25511.0,1,2021-09-10T00:26:31.100,12.0,TB,right,37.77,24.22,0.29,0.30,0.03,165.16,84.99,None,QB,TB,True
1,2021090900,97,25511.0,2,2021-09-10T00:26:31.200,12.0,TB,right,37.78,24.22,0.23,0.11,0.02,164.33,92.87,None,QB,TB,True
2,2021090900,97,25511.0,3,2021-09-10T00:26:31.300,12.0,TB,right,37.78,24.24,0.16,0.10,0.01,160.24,68.55,None,QB,TB,True
3,2021090900,97,25511.0,4,2021-09-10T00:26:31.400,12.0,TB,right,37.73,24.25,0.15,0.24,0.06,152.13,296.85,None,QB,TB,True
4,2021090900,97,25511.0,5,2021-09-10T00:26:31.500,12.0,TB,right,37.69,24.26,0.25,0.18,0.04,148.33,287.55,None,QB,TB,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069503,2021091300,4765,47906.0,43,2021-09-14T03:51:33.200,72.0,BAL,right,37.22,24.69,1.68,0.82,0.17,64.60,76.33,None,G,BAL,True
1069504,2021091300,4765,47906.0,44,2021-09-14T03:51:33.300,72.0,BAL,right,37.38,24.74,1.67,0.64,0.16,68.26,73.61,None,G,BAL,True
1069505,2021091300,4765,47906.0,45,2021-09-14T03:51:33.400,72.0,BAL,right,37.53,24.77,1.60,0.69,0.16,73.86,73.56,None,G,BAL,True
1069506,2021091300,4765,47906.0,46,2021-09-14T03:51:33.500,72.0,BAL,right,37.67,24.80,1.48,1.00,0.15,80.84,75.27,None,G,BAL,True


In [4]:
# Create separate dataframes for offense and defense
# 1a. Offense has possession flag set as True
o_df = player_tracking.loc[player_tracking['possession'] == True]
o_df = o_df[o_df['officialPosition'].isin(['G','C','T'])]

# 1b. Defense has possession flag set as False
x_df = player_tracking.loc[player_tracking['possession'] == False]
x_df = x_df.add_prefix('x_')

In [5]:
o_df

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,officialPosition,possessionTeam,possession
172,2021090900,97,40151.0,1,2021-09-10T00:26:31.100,66.0,TB,right,42.18,23.97,0.00,0.00,0.01,45.36,206.92,None,C,TB,True
173,2021090900,97,40151.0,2,2021-09-10T00:26:31.200,66.0,TB,right,42.16,23.97,0.00,0.00,0.02,45.36,216.20,None,C,TB,True
174,2021090900,97,40151.0,3,2021-09-10T00:26:31.300,66.0,TB,right,42.15,23.97,0.01,0.21,0.01,46.28,275.73,None,C,TB,True
175,2021090900,97,40151.0,4,2021-09-10T00:26:31.400,66.0,TB,right,42.15,23.98,0.06,0.63,0.01,46.28,308.29,None,C,TB,True
176,2021090900,97,40151.0,5,2021-09-10T00:26:31.500,66.0,TB,right,42.14,23.99,0.18,1.02,0.02,47.50,317.31,None,C,TB,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069503,2021091300,4765,47906.0,43,2021-09-14T03:51:33.200,72.0,BAL,right,37.22,24.69,1.68,0.82,0.17,64.60,76.33,None,G,BAL,True
1069504,2021091300,4765,47906.0,44,2021-09-14T03:51:33.300,72.0,BAL,right,37.38,24.74,1.67,0.64,0.16,68.26,73.61,None,G,BAL,True
1069505,2021091300,4765,47906.0,45,2021-09-14T03:51:33.400,72.0,BAL,right,37.53,24.77,1.60,0.69,0.16,73.86,73.56,None,G,BAL,True
1069506,2021091300,4765,47906.0,46,2021-09-14T03:51:33.500,72.0,BAL,right,37.67,24.80,1.48,1.00,0.15,80.84,75.27,None,G,BAL,True


In [6]:
x_df

,x_gameId,x_playId,x_nflId,x_frameId,x_time,x_jerseyNumber,x_team,x_playDirection,x_x,x_y,x_s,x_a,x_dis,x_o,x_dir,x_event,x_officialPosition,x_possessionTeam,x_possession
258,2021090900,97,41263.0,1,2021-09-10T00:26:31.100,90.0,DAL,right,43.66,18.70,0.45,0.92,0.05,308.77,337.59,None,DE,TB,False
259,2021090900,97,41263.0,2,2021-09-10T00:26:31.200,90.0,DAL,right,43.62,18.75,0.57,0.73,0.06,308.77,331.21,None,DE,TB,False
260,2021090900,97,41263.0,3,2021-09-10T00:26:31.300,90.0,DAL,right,43.59,18.80,0.63,0.48,0.06,308.77,331.38,None,DE,TB,False
261,2021090900,97,41263.0,4,2021-09-10T00:26:31.400,90.0,DAL,right,43.52,18.83,0.67,0.39,0.08,318.19,316.36,None,DE,TB,False
262,2021090900,97,41263.0,5,2021-09-10T00:26:31.500,90.0,DAL,right,43.41,18.87,0.85,1.00,0.12,311.63,298.40,None,DE,TB,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069456,2021091300,4765,43402.0,43,2021-09-14T03:51:33.200,44.0,LV,right,53.53,19.61,4.28,2.07,0.44,237.13,203.71,None,ILB,BAL,False
1069457,2021091300,4765,43402.0,44,2021-09-14T03:51:33.300,44.0,LV,right,53.34,19.24,4.04,3.09,0.41,241.46,209.58,None,ILB,BAL,False
1069458,2021091300,4765,43402.0,45,2021-09-14T03:51:33.400,44.0,LV,right,53.12,18.91,3.88,3.61,0.40,244.20,216.49,None,ILB,BAL,False
1069459,2021091300,4765,43402.0,46,2021-09-14T03:51:33.500,44.0,LV,right,52.87,18.63,3.68,4.27,0.38,245.61,225.43,None,ILB,BAL,False


In [7]:
# Merge the offense and defense dataframes
combined_df = pd.merge(o_df, x_df,  how='left', left_on=['gameId','playId','frameId'], right_on = ['x_gameId','x_playId', 'x_frameId'])
combined_df

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,...,x_y,x_s,x_a,x_dis,x_o,x_dir,x_event,x_officialPosition,x_possessionTeam,x_possession
0,2021090900,97,40151.0,1,2021-09-10T00:26:31.100,66.0,TB,right,42.18,23.97,...,18.70,0.45,0.92,0.05,308.77,337.59,None,DE,TB,False
1,2021090900,97,40151.0,1,2021-09-10T00:26:31.100,66.0,TB,right,42.18,23.97,...,32.69,0.00,0.00,0.01,265.97,215.94,None,DE,TB,False
2,2021090900,97,40151.0,1,2021-09-10T00:26:31.100,66.0,TB,right,42.18,23.97,...,29.62,0.25,0.26,0.03,264.41,162.64,None,MLB,TB,False
3,2021090900,97,40151.0,1,2021-09-10T00:26:31.100,66.0,TB,right,42.18,23.97,...,37.55,0.19,0.09,0.07,302.07,58.43,None,CB,TB,False
4,2021090900,97,40151.0,1,2021-09-10T00:26:31.100,66.0,TB,right,42.18,23.97,...,37.78,3.28,1.29,0.32,89.72,168.38,None,CB,TB,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2676471,2021091300,4765,47906.0,47,2021-09-14T03:51:33.600,72.0,BAL,right,37.81,24.84,...,22.73,2.56,3.80,0.24,21.81,37.92,None,CB,BAL,False
2676472,2021091300,4765,47906.0,47,2021-09-14T03:51:33.600,72.0,BAL,right,37.81,24.84,...,25.75,2.17,1.24,0.22,298.12,328.24,None,DT,BAL,False
2676473,2021091300,4765,47906.0,47,2021-09-14T03:51:33.600,72.0,BAL,right,37.81,24.84,...,21.14,5.36,3.77,0.53,297.76,286.08,None,FS,BAL,False
2676474,2021091300,4765,47906.0,47,2021-09-14T03:51:33.600,72.0,BAL,right,37.81,24.84,...,25.48,1.12,3.32,0.13,70.29,50.96,None,DE,BAL,False


In [10]:
combined_df['distance_opp'] = combined_df.apply(lambda row : math.sqrt(((row['x'] - row['x_x'])**2 + (row['y'] - row['x_y'])**2)), axis=1)
combined_df

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,...,x_s,x_a,x_dis,x_o,x_dir,x_event,x_officialPosition,x_possessionTeam,x_possession,diff_squared_sum
0,2021090900,97,40151.0,1,2021-09-10T00:26:31.100,66.0,TB,right,42.18,23.97,...,0.45,0.92,0.05,308.77,337.59,None,DE,TB,False,5.473874
1,2021090900,97,40151.0,1,2021-09-10T00:26:31.100,66.0,TB,right,42.18,23.97,...,0.00,0.00,0.01,265.97,215.94,None,DE,TB,False,8.914084
2,2021090900,97,40151.0,1,2021-09-10T00:26:31.100,66.0,TB,right,42.18,23.97,...,0.25,0.26,0.03,264.41,162.64,None,MLB,TB,False,6.142524
3,2021090900,97,40151.0,1,2021-09-10T00:26:31.100,66.0,TB,right,42.18,23.97,...,0.19,0.09,0.07,302.07,58.43,None,CB,TB,False,13.643039
4,2021090900,97,40151.0,1,2021-09-10T00:26:31.100,66.0,TB,right,42.18,23.97,...,3.28,1.29,0.32,89.72,168.38,None,CB,TB,False,14.003517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2676471,2021091300,4765,47906.0,47,2021-09-14T03:51:33.600,72.0,BAL,right,37.81,24.84,...,2.56,3.80,0.24,21.81,37.92,None,CB,BAL,False,3.514001
2676472,2021091300,4765,47906.0,47,2021-09-14T03:51:33.600,72.0,BAL,right,37.81,24.84,...,2.17,1.24,0.22,298.12,328.24,None,DT,BAL,False,2.106680
2676473,2021091300,4765,47906.0,47,2021-09-14T03:51:33.600,72.0,BAL,right,37.81,24.84,...,5.36,3.77,0.53,297.76,286.08,None,FS,BAL,False,11.795338
2676474,2021091300,4765,47906.0,47,2021-09-14T03:51:33.600,72.0,BAL,right,37.81,24.84,...,1.12,3.32,0.13,70.29,50.96,None,DE,BAL,False,1.195701


In [33]:
# Save to CSV
combined_df.to_csv('gs://big-data-bowl/tracking_with_opponent_week1.csv')